In [ ]:

from llama_index.readers.obsidian import ObsidianReader
import openai
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
from dotenv import load_dotenv

In [11]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [12]:
import nest_asyncio

nest_asyncio.apply()

In [13]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Build a simple directory

In [14]:
simple_glossary = SimpleDirectoryReader(
    r"/Users/arthursarazin/Documents/coreandgraphs/graphandgovern/graph"
).load_data()

# Upload Obsidian vault files

#### Using Obsidian reader from LLama Index

In [15]:
#locate obsidian files that make your graph
filepath = r"/Users/arthursarazin/Documents/coreandgraphs/graphandgovern/graph"

#load the graph files 
graph_files = ObsidianReader(filepath).load_data()
print(list(graph_files))

[Document(id_='d989c515-a572-4150-aa7c-b8f2f28c8094', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="---\nentite: base\n---\nCe document a pour objectif de décrire les aspects techniques du Datawarehouse et ses contrats d'interface avec le système ERP. Il inclut les spécifications des flux de données, les politiques d'accès, et les contraintes liées à l'utilisation.\n\n---\n", mimetype=None, path=None, url=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'), Document(id_='e6edc5ca-75a7-43b0-b8a0-1ba0ce7c1313', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='

# Using manual code to create specific ontology

In [16]:
# Fonction pour extraire les métadonnées YAML et le titre d'un fichier Markdown
def extract_metadata_and_title(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        # Recherchez la section YAML (entre `---` au début et à la fin)
        if content.startswith("---"):
            try:
                yaml_block = content.split("---", 2)[1]
                metadata = yaml.safe_load(yaml_block)
                return os.path.basename(file_path).replace(".md", ""), metadata
            except Exception as e:
                print(f"Erreur lors de l'extraction des métadonnées de {file_path}: {e}")
                return None, None
        return os.path.basename(file_path).replace(".md", ""), None

# Parcourir les fichiers Obsidian et créer des entités et relations
def process_obsidian_notes(directory):
    entities = []
    relations = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                title, metadata = extract_metadata_and_title(file_path)
                if title:
                    # Ajouter l'entité basée sur le titre
                    entities.append({"name": title})
                    
                    # Ajouter les relations basées sur les propriétés YAML
                    if metadata:
                        for key, value in metadata.items():
                            relations.append({
                                "type": key,  # Nom de la propriété comme relation
                                "source": title,  # Titre du fichier comme entité source
                                "target": value,  # Valeur de la propriété comme entité cible
                            })
    return entities, relations



In [17]:
# Exemple d'utilisation
obsidian_dir = r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\graph"
entities, relations = process_obsidian_notes(obsidian_dir)

print(relations)

[]


# Set LLM (OpenAI)

In [ ]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [19]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings ()

In [20]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [21]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [22]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or favorites. However, I can tell you about some popular data governance tools that are widely used in the industry. These include:

1. **Collibra**: Known for its comprehensive data governance platform that helps organizations manage data assets, ensure compliance, and improve data quality.

2. **Informatica Axon**: Offers a collaborative data governance solution that integrates with other Informatica products to provide a holistic view of data assets.

3. **Alation**: Provides a data catalog that helps organizations discover, understand, and govern their data assets effectively.

4. **Talend**: Offers data integration and governance solutions that help ensure d

In [23]:
# Instantiate vector store
simple_glossary = SimpleDirectoryReader(
    r"/Users/arthursarazin/Documents/coreandgraphs/graphandgovern/graph"
)

# Instantiate graph store 

In [24]:
graph_store = SimpleGraphStore()
graph_storage_context = StorageContext.from_defaults(graph_store=graph_store)

# Instantiate ontology store

In [25]:
onto_store = SimpleGraphStore()
onto_storage_context = StorageContext.from_defaults(graph_store=onto_store)

# Construct Index

## Construct Vector Store Index

In [26]:
vector_index = VectorStoreIndex.from_documents(
    documents = simple_glossary.load_data(),
    show_progress=True)

Parsing nodes:   0%|          | 0/111 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/112 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
# save index to disk
vector_index.set_index_id("vector_index")
vector_index.storage_context.persist("vector")

## Construct knowledge graph index 

In [42]:
def process_document(doc):
    # This function will be called for each document
    print(f"Processing document: {doc.doc_id}")
    return doc

In [ ]:
# Assuming you have your graph files loaded
print(f"Starting to process {len(graph_files)} documents...")

In [ ]:
# Wrap your document processing in a progress bar
graph_index = KnowledgeGraphIndex.from_documents(
        documents=[process_document(doc) for doc in graph_files],  # process one document at a time
        max_triplets_per_chunk=2,
        storage_context=graph_storage_context,
        include_embeddings=True,
    show_progress=True,
)


In [ ]:
print(graph_index)

In [46]:
graph_index.set_index_id("graph_index")
graph_index.storage_context.persist("knowledge_graph")
graph_storage_context.persist(persist_dir="knowledge_graph")


## Construct knowledge with ontology as assistant

In [47]:
onto_store = SimpleGraphStore()
onto_storage_context = StorageContext.from_defaults(graph_store=onto_store)

In [48]:
# Configurer DynamicLLMPathExtractor avec les entités et relations extraites
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=None,  # Si vous ne limitez pas les types d'entités
    allowed_relation_types=None,  # Si vous ne limitez pas les types de relations
    allowed_entity_props=None,
    allowed_relation_props=None,
)

In [ ]:
# Résumé des entités et relations extraites
print("Entités :")
for entity in entities:
    print(entity)

print("\nRelations :")
for relation in relations:
    print(relation)

In [ ]:
onto_index = PropertyGraphIndex.from_documents(
    documents=[process_document(doc) for doc in graph_files],
    llm=llm,
    storage_context=onto_storage_context,
    embed_kg_nodes=True,
    kg_extractors=[kg_extractor],
    show_progress=True,
    graph_store=onto_store 

)

In [ ]:
print(onto_storage_context)

In [52]:
# save index to disk
onto_index.set_index_id("onto_index")
onto_index.storage_context.persist("onto_graph")
onto_storage_context.persist(persist_dir="onto_graph")